# Solving LP problems with Gurobi library

https://www.gurobi.com/documentation/8.0/refman/index.html

Andrea Gasparin: andrea.gasparin@PHD.units.it

In [ ]:
# install xpress (the solvere we will use)
!pip install gurobipy
# import xpress (usually as xp)
import numpy as np
import gurobipy as gb

     |████████████████████████████████| 11.5 MB 4.3 MB/s 



#The assignment problem

A company has 4 machines available for the assignment to 4 tasks. Any
machine can be assigned to any task, but only one, and each task requires to be processed
by one machine only. The cost required to set up each machine for the
processing of each task is given in the table below.

 .|Task 1 |Task 2 |Task 3 |Task 4
-------|-------|-------|-------|------
Machine 1|13| 4| 7| 6
Machine 2| 1| 11 |5 |4
Machine 3| 6 |7 |2 |8
Machine 4| 1| 3| 5| 9

The company wants to minimise the total setup cost needed for the
processing of all four tasks.

It is intuitive to assign the decision variables as:

$$
x_{ij} = \begin{cases} &1 \quad \text{if machine $i$ is assigned to process task $j$}\\
&0 \quad \text{if machine $i$ is not assigned to process task $j$}
\end{cases}
$$
with $i = 1, 2, 3, 4$ and $j = 1, 2, 3, 4$.

The problem can be then formulated as follow:


$
\begin{gather}
\min 13x_{11} + 4x_{12} + 7x_{13} + 6x_{14}+
x_{21} + 11x_{22} + 5x_{23} + 4x_{24}+
6x_{31} + 7x_{32} + 2x_{33} + 8x_{34}+
x_{41} + 3x_{42} + 5x_{43} + 9x_{44}\\
x_{11} + x_{12} + x_{13} + x_{14} \leq 1\\
x_{21} + x_{22} + x_{23} + x_{24} \leq 1\\
x_{31} + x_{32} + x_{33} + x_{34} \leq 1\\
x_{41} + x_{42} + x_{43} + x_{44} \leq 1\\
x_{11} + x_{21} + x_{31} + x_{41} = 1\\
x_{12} + x_{22} + x_{32} + x_{42} = 1\\
x_{13} + x_{23} + x_{33} + x_{43} = 1\\
x_{14} + x_{24} + x_{34} + x_{44} = 1\\
x_{ij} \in \{0, 1\},\quad for\quad i = 1, . . . , 4,\; j = 1, . . . , 4
\end{gather}
$

Equivalently, defining 

$$
c_{ij} = \text{cost of machine $i$ to perform task $j$}\\
I = \{1,2,3,4\}\\
J = \{1,2,3,4\}
$$

we can rewrite the problem in a more compact way as:

$
\begin{gather}
\min \sum_{i\in I,j\in J} c_{ij}x_{ij}\\
∑_{j\in J} x_{ij}  \leq 1 \quad \forall i \in I\\
∑_{i\in I} x_{ij}  = 1 \quad \forall j \in J\\
x_{ij} \in \{0, 1\},\quad for\quad i \in I,\; j \in J
\end{gather}
$

In [ ]:
assignment = gb.Model()
assignment.modelSense = gb.GRB.MINIMIZE #declare mimization

# assignment.setParam('OutputFlag', 0) suppress outputs, equivalent of xpress setControl('outputlog', 0)



X = assignment.addVars( [(i,j) for i in range(4) for j in range(4)], vtype=gb.GRB.BINARY) #this way of declare vars does not allow to work with matrix multiplication (X is a tuple dict)

I = range(4)
J = range(4)

costs = np.array([[13,	4,	7, 6],
                	[1,	11,	5, 4],
                  [6,	7,	2, 8],
                  [1,	3,	5, 9]])


for i in I:
  assignment.addConstr( gb.quicksum(X[i,j] for j in J) <= 1) #quicksum is the equivalent to xp.Sum

for j in J:
  assignment.addConstr( gb.quicksum(X[i,j] for i in I) == 1 )

assignment.setObjective( 
    gb.quicksum( costs[i,j]*X[i,j]   for j in J for i in I)
    )   

assignment.optimize() #equivalent to solve() for xpress


print( "\n", type(X), X, "\n")
print("\nSolution")

for i in I:
  for j in J:
    if X[i,j].x == 1:  #to access the variable value 
      print("Machine ", i, " is assigned to task ", j)

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 8 rows, 16 columns and 32 nonzeros
Model fingerprint: 0x6f06b897
Variable types: 0 continuous, 16 integer (16 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 24.0000000
Presolve time: 0.00s
Presolved: 8 rows, 16 columns, 32 nonzeros
Variable types: 0 continuous, 16 integer (16 binary)

Root relaxation: objective 1.100000e+01, 6 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0      11.0000000   11.00000  0.00%     -    0s

Explored 1 nodes (6 simplex iterations) in 0.04 seconds (0.00 work units)
Thread count w



 Solution:
 [[-0.  1. -0. -0.]
 [-0. -0. -0.  1.]
 [-0. -0.  1. -0.]
 [ 1. -0. -0. -0.]] 

#Set covering


A typical application concerns facility location. Suppose we are given a
set of potential sites $N = \{1, . . . , n\}$ for the location of fire stations. A
station placed at $j$ costs $c_j$ . We are also given a set of communities
$M = \{1, . . . , m\}$ that have to be protected. The subset of communities
that can be protected from a station located at $j$ is $M_j$ . For example,
$M_j$ might be the set of communities that can be reached from $j$ within 10
minutes. Then the problem of choosing a minimum-cost set of locations
for the fire stations such that each community can be reached from some
fire station in 10 minutes is a set-covering problem.

### Decision variables

$$
x_{j} = \begin{cases} &1 \quad \text{if} \; \text{a station is placed in $j$}\\
&0 \quad \text{otherwise}
\end{cases}
$$

### Obj

$$
\min \;\sum_{j=1}^n c_j x_{j}
$$

###Constraints 
Let A be the $m \times n$ incidence matrix of the family $\{M_j\}$ for $j \in N$, that is, for $i \in M$,

$$
a_{ij} =\begin{cases} &1 \quad \text{if}\; i \in M_j\\
&0 \quad \text{otherwise}
\end{cases}
$$
A solution $x \in \{0, 1\}^n$ covers all sites if and only if  satisfies
$$
Ax ≥ \overline{1}
$$
where $\overline{1}$ is a $m$−vector all of whose components are equal to 1.


###In this case
Let 
$$
M = \{1, 2, 3, 4, 5\}, \\
\{M\} = \{\{1, 3, 5\}, \{2, 3\}, \{1, 4\}, \{3, 4, 5\}, \{2\}, \{5\}, \{1, 5\}\},\\
c = [3, 5, 1, 9, 2, 4, 1].
$$
 Hence,
$$
A=
\begin{pmatrix}
1 &0 &1 &0 &0 &0 &1\\
0 &1 &0 &0 &1 &0 &0\\
1 &1 &0 &1 &0 &0 &0\\
0 &0 &1 &1 &0 &0 &0\\
1 &0 &0 &1 &0 &1 &1\\
\end{pmatrix}
$$

In [ ]:
set_covering= gb.Model()
set_covering.setParam('outputFlag', 0)
set_covering.modelSense = gb.GRB.MINIMIZE

costs = np.array([3,5,1,9,2,4,1])

A = np.array([[1, 0, 1, 0, 0, 0, 1],
              [0, 1, 0, 0, 1, 0, 0],
              [1, 1, 0, 1, 0, 0, 0],
              [0, 0, 1, 1, 0, 0, 0],
              [1, 0, 0, 1, 0, 1, 1]])
b = np.ones(5)
x = set_covering.addMVar(7, vtype=gb.GRB.BINARY) # this allow mat operations, very similar (amd related) to np.arrays

set_covering.addConstr( A @ x >= b )  # @ is the equivalent of xp.Dot
set_covering.setObjective( costs @ x)

set_covering.optimize()
print(set_covering.status) # https://www.gurobi.com/documentation/9.5/refman/optimization_status_codes.html for all codes

print( "optimal" if set_covering.status == 2 else ("infeasible" if set_covering.status == 3 else ("unbounded" if set_covering.status == 5 else "check the link page for other status codes")))


print("\n\n",x.x, type(x), type(x.x))

2
optimal


 [1. 0. 1. 0. 1. 0. 0.] <class 'gurobipy.MVar'> <class 'numpy.ndarray'>


## Parameter tuning

for all params:

https://www.gurobi.com/documentation/9.5/refman/parameters.html

In [29]:
print(set_covering.getParamInfo("timeLimit"))


('timeLimit', <class 'float'>, inf, 0.0, inf, inf)


In [34]:

print(set_covering.setParam('TimeLimit', 5*60), "\n")

print(set_covering.getParamInfo("TimeLimit"))

print(set_covering.params.TimeLimit)


None 

('TimeLimit', <class 'float'>, 300.0, 0.0, inf, inf)
300.0
